In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from time import sleep
import random
soup_list = []
page = 0
while True:
    
    service_obj = Service("./drivers/chromedriver")
    driver = webdriver.Chrome(service=service_obj)
    url = 'https://www.mycareersfuture.sg/search?sortBy=new_posting_date&page='+str(page)

    #search=research%20scientist&sortBy=new_posting_date&page=2'

    driver.get(url)
    sleep(2)
    html = driver.page_source
    soup_list.append(BeautifulSoup(html, 'lxml'))
    #print(soup.prettify())

    driver.close()
    page+=1
    if (page == 1000):
        break
    

WebDriverException: Message: unknown error: net::ERR_ADDRESS_UNREACHABLE
  (Session info: chrome=111.0.5563.110)
Stacktrace:
#0 0x55dd948c1243 <unknown>
#1 0x55dd946857a6 <unknown>
#2 0x55dd9467d728 <unknown>
#3 0x55dd94670772 <unknown>
#4 0x55dd94671e82 <unknown>
#5 0x55dd94670b3a <unknown>
#6 0x55dd9466fbf8 <unknown>
#7 0x55dd9466fa30 <unknown>
#8 0x55dd9466e508 <unknown>
#9 0x55dd9466eb3d <unknown>
#10 0x55dd94687596 <unknown>
#11 0x55dd946fbfc5 <unknown>
#12 0x55dd946e3082 <unknown>
#13 0x55dd946fb932 <unknown>
#14 0x55dd946e2e53 <unknown>
#15 0x55dd946b59ea <unknown>
#16 0x55dd946b6b2e <unknown>
#17 0x55dd94915d5e <unknown>
#18 0x55dd94919a80 <unknown>
#19 0x55dd948fb8b0 <unknown>
#20 0x55dd9491ab63 <unknown>
#21 0x55dd948ecf75 <unknown>
#22 0x55dd9493d998 <unknown>
#23 0x55dd9493db27 <unknown>
#24 0x55dd94958c23 <unknown>
#25 0x7f0cd8894b43 <unknown>


In [5]:
# Initialized all variables with empty lists
job_title = []
company_name = []
job_location = []
job_level = []
salary = []
salary_min = []
salary_max = []
salary_type = []
days_posted = []
job_desc = []
job_req = []
job_type = []
job_cat = []
job_url = []
job_count = 0
jip_count = 0
is_responsive = []
is_contract = []
is_intern = []
is_temp = []
is_perm = []
link_list = []    # initialise list to hold the links of job postings
job_class_list = []
timer = random.randint(2, 5)

minreq = 15
print(len(soup_list))


889


In [7]:
import pandas as pd
import numpy as np
# Get the links of job postings on the results page for the job listings
#class_list = soup.find_all('a', {'class': 'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'})
# initialise dataframe to hold the url list 
data_df = pd.DataFrame()
for soup in soup_list:
    
    list_of_jobs = soup.findAll("p", {'data-testid':'company-hire-info'})
    for x in range(len(list_of_jobs)):
        company_name.append(list_of_jobs[x].text)

    # Job Title
    job_title_var = soup.findAll("span", {'data-cy':'job-card__job-title'})
    for x in range(len(job_title_var)):
        job_title.append(job_title_var[x].text)

    # Job Links
    job_link_var = soup.findAll('a', {'data-testid':'job-card-link'})     
    for x in range(len(job_link_var)):
        job_class_list.append(job_link_var[x]["href"])
        job_url.append("https://www.mycareersfuture.sg"+job_link_var[x]["href"])   


    #print('==== List of job postings url ====')
    #print('='*35)        
    #for i in range(len(list_of_jobs)):
    #    print("Company: " + company_name[i])
    #    print("Job Title: " + job_title[i])
    #    print("Job URL: " + job_url[i])
        #print("job class: " + job_class_list[i]["href"])
i=0
for job in list_of_jobs:    
    job_info = [{
        'company' : job.get_text(),
        'job_title' : job_title[i],
        'url' : job_url[i]
    }]
    i+=1

    # append the job posting to dataframe
    temp_df = pd.DataFrame(job_info)
    data_df = pd.concat([data_df, temp_df], axis=0)
data_df.to_csv('job_info_list.csv', encoding='utf-8')
    

In [8]:
print(len(job_url))

17780


In [9]:

i=0
# iterate through the link_list to load each job posting
for url in job_url:
#    if (i>10):
#        break
            
    driver_job = webdriver.Chrome(executable_path="./drivers/chromedriver")
    driver_job.get(url)
    sleep(1)
    posting = driver_job.page_source
    
    job_soup = BeautifulSoup(posting, 'lxml')
    #print(job_soup.prettify())
    
    # get company address 
    job_location_var = job_soup.find('p', {'id':'address'})
    if (job_location_var):
        job_location.append(job_location_var.get_text())
    else:
        job_location.append("Unknown") 

    # Job type 
    job_level_var = job_soup.find("p", {'id':'employment_type'})
    if (job_level_var):
        job_type.append(job_level_var.get_text())
    else:
        job_type.append("Unknown")

    # Job Seniority (if any?)
    job_level_var = job_soup.find("p", {'id':'seniority'})
    if (job_level_var):
        job_level.append(job_level_var.get_text())
    else:
        job_level.append("Unknown")

    # Job Category
    job_level_var = job_soup.find("p", {'id':'job-categories'})
    if (job_level_var):
        job_cat.append(job_level_var.get_text())
    else:
        job_cat.append("Unknown")

    # find salary 
    salary_range = job_soup.find_all('div', {'class':'lh-solid'})
    for element in salary_range:
        salary.append(element.get_text())
    if (salary):
        salary_list = salary[i].split("to")
        if (len(salary_list) == 1):
            salary_min.append(salary_list[0])
            salary_max.append(salary_list[0])
        elif (len(salary_list) == 2):
            salary_min.append(salary_list[0])
            salary_max.append(salary_list[1])
        
    # find salary type
    job_info_list = job_soup.find('span', {'data-testid' : 'salary-type'})
    if (job_info_list):
        salary_type.append(job_info_list.get_text())
    else:
        salary_type.append("Unknown")

    # getting the job description
    job_desc_section = job_soup.find('div', {'id':'description-content'})
    if (job_desc_section):
        job_desc.append(job_desc_section.get_text())
    else:
        job_desc.append("Unknown") 
 
    # getting the job description
    job_req_section = job_soup.find('div', {'id':'skills-needed'})
    if (job_req_section):
        label_list = job_req_section.contents[1].contents
        label_str = str()
        j=0
        for j in range(len(label_list)):
            if (j>0):
                label_str=label_str + "|" + label_list[j].get_text() 
        
    if (label_str):
        job_req.append(label_str)
    else:
        job_req.append("Unknown") 
        
    driver_job.close
    i+=1


/tmp/ipykernel_6244/2418609643.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_job = webdriver.Chrome(executable_path="./drivers/chromedriver")


In [12]:


i=0  
# initialise dataframe to hold the scrapping results
data_df = pd.DataFrame()

for url in job_url:
#    if (i>10):
#        break
    #print("Company: " + job.get_text())
    #print("Job Title: " + job_title[i])
    #print("Job URL: " + job_url[i])
    #print("Job Location: " + job_location[i])
    #print("Job Type: " + job_type[i])
    #print("Job seniority: " + job_level[i])
    #print("Job Category: " + job_cat[i])
    #print("Salary: " + salary_min[i] +" to " + salary_max[i])
    #print("Salary type: " + salary_type[i])
    #print ("Desc: " + job_desc[i])
    #print ("Req: " + job_req[i] +"\n")
 
    # Process job description


    job_posting = [{
        'company' : company_name[i],
        'job_title' : job_title[i],
        'location' : job_location[i],
        'min salary' : salary_min[i],
        'max salary' : salary_max[i],
        'salary_type' : salary_type[i],
        'job_description' : job_desc[i],
        'job_requirement' : job_req[i],
        'url' : url
    }]
    i+=1
    
    # append the job posting to dataframe
    temp_df = pd.DataFrame(job_posting)
    data_df = pd.concat([data_df, temp_df], axis=0)
        


In [13]:
data_df.shape


(17780, 9)

In [14]:
data_df.to_csv('data_wh_ft.csv', encoding='utf-8')